# Setup: Test & Explore

Start here!
This file is used to test and explore the functions and get a better understanding of **prompt engineering, vector databases & the whole Retrieval-augmented generation (RAG) architecture**.

In [1]:
# important for changing code in the modules without restarting the Kernel
%load_ext autoreload
%autoreload 2

# import local functions
from TalkToOllama import LLMOllama # --> TalkToOllama.py
from TalkToDatabase import VectorDB # --> TalkToDatabase.py
from ImportData import DataHandler # --> ImportData.py

# Logging
import logging
log = logging.getLogger("applogger")
logging.basicConfig(level=logging.INFO)

## Connection & Instances

In [2]:
vectordb = VectorDB(
  collection_name="catcenter_vectors",
  database_path="chromadb/"
)

LLM = LLMOllama(
  database=vectordb,
  model="llama3"
)

datahandler = DataHandler(
  database=vectordb
)

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


## Import Data to vector database

In [3]:
# Scrape data from web-docs
datahandler.scrape_apidocs_catcenter()

# Import from Specfication file
datahandler.import_apispecs_from_json()

INFO:applogger:['Cisco Catalyst Center Platform APIs and Integrations Overview. - Cisco Catalyst Center 2.3.7 - Cisco DevNet                          Cisco Catalyst Center Platform APIs and Integrations Overview. This page explains Catalyst Center integration options: Intent APIs provide REST access to the Catalyst Center platform. Integration APIs enable IT Service Management workflows. Events and Notifications WebHooks enables notification on key events. The Multivendor API enables direct support for non-Cisco devi', 'ces. Cisco Catalyst Center Platform Overview Cisco Catalyst Center is at the heart of Cisco’s intent-based network architecture. Catalyst Center supports the expression of business intent for network use cases, such as base automation capabilities in the enterprise network. The Analytics and Assurance features of Catalyst Center provide end-to-end visibility into the network with full context through data and insights. Cisco customers and partners can use the Catalyst C

## Understand Tokenization & Embeddings

In [14]:
from sentence_transformers import SentenceTransformer

# all-MiniLM-L6-v2 generates vectors with 384 dimensions
model = SentenceTransformer("all-MiniLM-L6-v2") 

# Encoding text
text = "Cisco Catalyst Center is at the heart of Cisco’s intent-based network architecture. Catalyst Center supports the expression of business intent for network use cases, such as base automation capabilities in the enterprise network. The Analytics and Assurance features of Catalyst Center provide end-to-end visibility into the network with full context through data and insights."
embedding = model.encode(text,output_value=None)

print("Text:", text)
print("Token IDs:", embedding["input_ids"])
print("Embedding:", embedding["sentence_embedding"])

/Users/flpachin/Library/Caches/pypoetry/virtualenvs/understanding-rag-catalyst-center-VxZhv8nm-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
/Users/flpachin/Library/Caches/pypoetry/virtualenvs/understanding-rag-catalyst-center-VxZhv8nm-py3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	

Text: Cisco Catalyst Center is at the heart of Cisco’s intent-based network architecture. Catalyst Center supports the expression of business intent for network use cases, such as base automation capabilities in the enterprise network. The Analytics and Assurance features of Catalyst Center provide end-to-end visibility into the network with full context through data and insights.
Token IDs: tensor([  101, 26408, 16771,  2415,  2003,  2012,  1996,  2540,  1997, 26408,
         1521,  1055,  7848,  1011,  2241,  2897,  4294,  1012, 16771,  2415,
         6753,  1996,  3670,  1997,  2449,  7848,  2005,  2897,  2224,  3572,
         1010,  2107,  2004,  2918, 19309,  9859,  1999,  1996,  6960,  2897,
         1012,  1996, 25095,  1998, 16375,  2838,  1997, 16771,  2415,  3073,
         2203,  1011,  2000,  1011,  2203, 16476,  2046,  1996,  2897,  2007,
         2440,  6123,  2083,  2951,  1998, 20062,  1012,   102],
       device='mps:0')
Embedding: tensor([-1.5361e-02,  1.0563e-02, -5.6

## Explore Vectore Database

In [4]:
#The total number of embeddings added to the database:
vectordb.collection.count()

1019

In [7]:
# similarity search with the defined keyword
vectordb_query = vectordb.collection.query(
        query_texts=["eox status"],
        n_results = 3,
        include = ["embeddings", "metadatas", "documents", "distances"]
)
vectordb_query

{'ids': [['getEoXSummary_0',
   'getEoXStatusForAllDevices_0',
   'getEoXDetailsPerDevice_0',
   'reports_14',
   'ciscoISEServerIntegrationStatus_0']],
 'distances': [[0.8673032522201538,
   0.9057649970054626,
   0.9375768899917603,
   0.9783887267112732,
   1.3661105632781982]],
 'metadatas': [[{'doc_type': 'apispecs',
    'summary': 'Get EoX Summary',
    'tag': 'EoX'},
   {'doc_type': 'apispecs',
    'summary': 'Get EoX Status For All Devices',
    'tag': 'EoX'},
   {'doc_type': 'apispecs',
    'summary': 'Get EoX Details Per Device',
    'tag': 'EoX'},
   {'doc_type': 'apidocs'},
   {'doc_type': 'apispecs',
    'summary': 'Cisco ISE Server Integration Status',
    'tag': 'System Settings'}]],
 'embeddings': [[[-0.045885831117630005,
    0.08671378344297409,
    0.040732935070991516,
    -0.029681213200092316,
    0.027552831918001175,
    -0.016368284821510315,
    -0.011157115921378136,
    0.05888548865914345,
    -0.062204066663980484,
    -0.055864475667476654,
    0.01859752

In [8]:
# Total number of characters in the first document
# Should not exceed the defined chunking size
len(vectordb_query["documents"][0][0])

1600

## Talk to LLM

Prompting guide: https://llama.meta.com/docs/how-to-guides/prompting/ 

In [5]:
LLM.ask_llm("What is Cisco?")

2024-05-24 19:21:24 - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


{'model': 'llama3',
 'created_at': '2024-05-24T17:21:24.881694Z',
 'message': {'role': 'assistant',
  'content': "Cisco Systems, Inc. is an American technology company that specializes in designing and selling networking equipment, telecommunications equipment, and other high-technology services. The company was founded in 1984 by Leonard Bosack and Sandy Lerner.\n\nCisco is best known for its routers and switches, which are used to connect and manage computer networks, including the Internet. They also produce a wide range of other products, such as:\n\n1. Network devices: Routers, switches, firewalls, and network interface cards (NICs).\n2. Telepresence systems: Virtual meeting platforms that allow remote participants to interact with each other.\n3. Video conferencing solutions: Software-based video conferencing tools for businesses.\n4. Security appliances: Firewalls, intrusion detection systems, and virtual private networks (VPNs).\n5. Data center infrastructure: Servers, storage 

In [13]:
ask = LLM.ask_llm_rag(
    query_string="eox status of all devices",
    n_results=5
)
print(ask)

INFO:applogger:=== USER PROMPT === 
 User question: 'eox status of all devices'
    Context delimited with XML tags: <context>[['The provided REST API query path "/dna/intent/api/v1/eox-status/summary" with the operation "get" is used to retrieve the End of Sale (EoX) summary for all devices in the network. This query enables users to access a consolidated overview of the EoX status of all devices present in their network.  From a user perspective, this query is valuable for network administrators, operators, and IT personnel responsible for managing and maintaining network infrastructure. By making this query, users can easily obtain critical information about the EoX status of all devices in their network. This information can include details such as upcoming end-of-life dates for hardware and software components, recommended upgrades or replacements, and any relevant notices issued by Cisco regarding the devices.  By utilizing this query, users can stay proactive in planning for nec

User question: "eox status of all devices"

Based on the context, the right REST API call for this user question is:

`GET /dna/intent/api/v1/eox-status/summary`

This query provides a consolidated overview of the End-of-Sale (EoX) status of all devices in the network. 

Here are the required steps and documentation:

* Authentication: 
    * First, you need to obtain an access token by calling the REST API query `/dna/system/api/v1/auth/token` using Basic authentication scheme as defined in RFC 7617.
    * Set the `X-Auth-Token` header parameter with the obtained access token for all subsequent API queries.

Here is the source code for this user question in Python:
```python
import requests

# Obtain an access token by calling the REST API query '/dna/system/api/v1/auth/token'
auth_response = requests.post('https://your_cisco_dna_url.com/dna/system/api/v1/auth/token', auth=('your_username', 'your_password'))

# Set the 'X-Auth-Token' header parameter with the obtained access token
acc